In [ ]:
# For development purposes, reload imported modules when source changes
%load_ext autoreload
%autoreload 2

import spikeextractors as se
import spikewidgets as sw
import spiketoolkit as st
import os, sys

In [ ]:
# Create a toy example dataset
recording, sorting_true=sw.example_datasets.toy_example1(duration=60)

In [ ]:
# View the filtered timeseries data
filt=st.preprocessing.bandpass_filter(recording,freq_min=300,freq_max=6000)
sw.TimeseriesWidget(recording=filt,width=7,height=2.5).display()

In [ ]:
# Create a temporary working directory if needed
if not os.path.exists('tmp'): os.mkdir('tmp')

In [ ]:
# Mountainsort4 spike sorting
sorting_MS4 = st.sorters.run_mountainsort4(
    recording=recording,
    detect_sign=-1,
    adjacency_radius=-1
)

In [ ]:
# SpyKING Circus spike sorting
sorting_SC = st.sorters.run_spykingcircus(recording,
    adjacency_radius=50,
    detect_sign=-1,
    output_folder='tmp_SC',
)

In [ ]:
# KiloSort spike sorting (KILOSORT_PATH and NPY_MATLAB_PATH can be set as environment variables)
sorting_KS = st.sorters.run_kilosort(recording,
    output_folder='tmp_KS'
)

In [ ]:
# Klusta spike sorting
sorting_KL = st.sorters.run_klusta(recording,
        output_folder='tmp_KL'
        )

In [ ]:
# IronClust spike sorting (IRONCLUST_PATH can be set as environment variables)
sorting_IC = st.sorters.run_ironclust(recording,
    detect_sign=-1,
    adjacency_radius=-1,
    prm_template_name='tetrode_template.prm',
    ironclust_path=None
)

In [ ]:
# Tridesclous
sortin_TDC = st.sorters.run_tridesclous(recording, output_folder='tmp_TDC')


In [ ]:
sorting_true.getUnitIds()

In [ ]:
# Compute SNRs of true units
snrs=st.computeUnitSNR(recording=filt, sorting=sorting_true, unit_ids=sorting_true.getUnitIds(), max_num_waveforms=100)
sorting_true.setUnitsProperty(property_name='SNR',values=snrs)

In [ ]:
# Comparison MountainSort4 vs truth
comparison_MS4=st.comparison.SortingComparison(sorting_true,sorting_MS4)
sw.SortingAccuracyWidget(sorting_comparison=comparison_MS4, property_name='SNR').plot()

In [ ]:
# Comparison KiloSort vs truth
comparison_KS=st.comparison.SortingComparison(sorting_true,sorting_KS)
sw.SortingAccuracyWidget(sorting_comparison=comparison_KS, property_name='SNR').plot()

In [ ]:
# Comparison SpyKING Circus vs truth
comparison_SC=st.comparison.SortingComparison(sorting_true,sorting_SC)
sw.SortingAccuracyWidget(sorting_comparison=comparison_SC, property_name='SNR').plot()

In [ ]:
# Comparison Klusta vs truth
comparison_KL=st.comparison.SortingComparison(sorting_true,sorting_KL)
sw.SortingAccuracyWidget(sorting_comparison=comparison_KL, property_name='SNR').plot()

In [ ]:
# Comparison IronClust vs truth
comparison_IC=st.comparison.SortingComparison(sorting_true,sorting_IC)
sw.SortingAccuracyWidget(sorting_comparison=comparison_IC, property_name='SNR').plot()

In [ ]:
# Comparison tridesclous vs truth
comparison_TDC = st.comparison.SortingComparison(sorting_true,sorting_TDC)
sw.SortingAccuracyWidget(sorting_comparison=comparison_IC, property_name='SNR').plot()

In [ ]:
sw.SortingComparisonTable(comparison_MS4).display()

In [ ]:
sw.SortingComparisonTable(comparison_KS).display()

In [ ]:
sw.SortingComparisonTable(comparison_SC).display()

In [ ]:
sw.SortingComparisonTable(comparison_KL).display()

In [ ]:
sw.SortingComparisonTable(comparison_IC).display()

In [ ]:
sw.SortingComparisonTable(comparison_TDC).display()

In [ ]:
comparison_KL.plotConfusionMatrix()

In [ ]:
# You can use Phy to manually curate the sorting output of any spike sorter
st.exportToPhy(recording, sorting_MS4, output_folder='mountainsort')

In [ ]:
sorting_M4_curated = si.PhysortingExtractor('mountainsort/')